## Introduction

This notebook just covers the process of loading, compiling, and cleaning raw IPEDS data for a project.

## Packages & Data

Loading data and necessary packages.

In [43]:
## loading packages
import pandas            as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute       import IterativeImputer
from sklearn.tree         import DecisionTreeRegressor


## loading data
df_directory   = pd.read_csv("raw data\\hd_compiled.csv", encoding = "ISO-8859-1")
df_admissions  = pd.read_csv("raw data\\adm_compiled.csv", encoding = "ISO-8859-1")
df_fin_aid     = pd.read_csv("raw data\\sfa_compiled.csv", encoding = "ISO-8859-1")
df_enrollment  = pd.read_csv("raw data\\efa_compiled.csv", encoding = "ISO-8859-1")
df_grad_rate   = pd.read_csv("raw data\\gr200_compiled.csv", encoding = "ISO-8859-1")
df_migration   = pd.read_csv("raw data\\efc_compiled.csv", encoding = "ISO-8859-1")
df_finance     = pd.read_csv("raw data\\f_f1a_compiled.csv", encoding = "ISO-8859-1")
df_charges     = pd.read_csv("raw data\\ic_ay_compiled.csv", encoding = "ISO-8859-1")
df_misc        = pd.read_csv("raw data\\efd_compiled.csv", encoding = "ISO-8859-1")


## Directory Information

Limiting the dataset to active, public universities that offer an undergradaute and graduate degree. Also filtering out any tribal or historically black colleges and universities. 

In [44]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_directory = (df_directory
                ## Filtering rows
                .loc[(df_directory["ACAD_YEAR"]                                == 2023)     &
                     (df_directory["Institution is active in current year"]    == "Yes")    & 
                     (df_directory["Tribal college"]                           == "No")     &
                     (df_directory["Historically Black College or University"] == "No")     &
                     (df_directory["Control of institution"]                   == "Public") &
                     (df_directory["Degree-granting status"]                   == "Degree-granting") & 
                     (df_directory["Undergraduate offering"]                   == "Undergraduate degree or certificate offering") &
                     (df_directory["Graduate offering"]                        == "Graduate degree or certificate offering")]
                ## Subsetting columns
                [["UNITID",
                  "Institution (entity) name",
                  "City location of institution", 
                  "State abbreviation_value",
                  "State abbreviation"]]
                ## Renaming columns
                .rename(columns = {"UNITID"                       : "INST_ID",
                                   "Institution (entity) name"    : "INST_NAME",
                                   "City location of institution" : "INST_CITY",
                                   "State abbreviation_value"     : "INST_STATE_ABRV",
                                   "State abbreviation"           : "INST_STATE_NAME"})
)


## Admissions Information

Obtaining the number of applications, admits, and enrolled students - so that I can calculate admit rate and yield. Admit rate is a proxy for selectivity, while yield is the percentage of admitted students who enroll and a common metric in enrollment management.

In [45]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_admissions = (df_admissions
                 ## Filtering rows
                 .loc[(df_admissions["ACAD_YEAR"] == 2023) & 
                      (df_admissions["UNITID"].isin(df_directory["INST_ID"]))]
                 ## Subsetting columns
                 [["UNITID",
                   "Applicants total",
                   "Admissions total",
                   "Enrolled total"]]
                 ## Renaming columns 
                 .rename(columns = {"UNITID"           : "INST_ID",
                                    "Applicants total" : "APPLY_CT",
                                    "Admissions total" : "ADMIT_CT",
                                    "Enrolled total"   : "ENROLL_CT"})
)


## feature creation
df_admissions = (df_admissions
                 .assign(ADMIT_RATE = round(df_admissions["ADMIT_CT"]/df_admissions["APPLY_CT"]*100, 0),
                         YIELD_RATE = round(df_admissions["ENROLL_CT"]/df_admissions["ADMIT_CT"]*100, 0))
                 .drop(["APPLY_CT", "ADMIT_CT", "ENROLL_CT"], axis = 1)
)


## Financial Aid

Obtaining the percentage of freshmen who receive a Pell Grant or institutional grant aid (i.e. scholarships). I'm also getting the percentage of freshmen who live on campus, which is a proxy for identifying commuter schools. 

In [46]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_fin_aid = (df_fin_aid
              ## Filtering rows
              .loc[(df_fin_aid["ACAD_YEAR"] == 2023) & 
                   (df_fin_aid["UNITID"].isin(df_directory["INST_ID"]))]
              ## Subsetting columns
              [["UNITID",
                "Percent of full-time first-time undergraduates awarded Pell grants",
                "Percent of full-time first-time undergraduates awarded institutional grant aid",
                "Number living on-campus, (current year)(GISTON2_70501)",
                "Total number, (current year)(GISTN2_70496)"]]
              ## Renaming column
              .rename(columns = {"UNITID"                                                                         : "INST_ID",
                                 "Percent of full-time first-time undergraduates awarded Pell grants"             : "FINAID_PELL_GRANT_PCT",
                                 "Percent of full-time first-time undergraduates awarded institutional grant aid" : "FINAID_INST_AID_PCT",
                                 "Number living on-campus, (current year)(GISTON2_70501)"                         : "FINAID_ON_CAMPUS",
                                 "Total number, (current year)(GISTN2_70496)"                                     : "FINAID_COHORT"})
)


## feature creation
df_fin_aid = (df_fin_aid
              .assign(ON_CAMPUS_PCT = round(df_fin_aid["FINAID_ON_CAMPUS"]/df_fin_aid["FINAID_COHORT"]*100, 0))
              .drop(["FINAID_ON_CAMPUS", "FINAID_COHORT"], axis = 1)
)


## Enrollment

Obtaining the number graduate students, degree-seeking undergraduates, and non-degree seeking undergraduates. Non-degree seeking undergraduates would include high school students earning concurrent enrollment.

In [47]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_enrollment = (df_enrollment
                 ## Filtering rows
                 .loc[(df_enrollment["ACAD_YEAR"] == 2023)                    &
                      (df_enrollment["UNITID"].isin(df_directory["INST_ID"])) &
                      (df_enrollment["Level of student(EFALEVEL_20166)"].isin(["All students, Graduate",
                                                                               "All students, Undergraduate, Degree/certificate-seeking total",
                                                                               "All students, Undergraduate, Non-degree/certificate-seeking"]))]
                  ## Subsetting columns
                  [["UNITID",
                    "Level of student(EFALEVEL_20166)",
                    "Grand total(EFTOTLT_20286)"]]
                  ## Reshaping data
                  .pivot(index   = "UNITID",
                         columns = "Level of student(EFALEVEL_20166)",
                         values  = "Grand total(EFTOTLT_20286)")
                  .reset_index()
                  ## Renaming columns
                  .rename(columns = {"UNITID"                                                              : "INST_ID",
                                     "All students, Graduate"                                              : "ENROLL_GRAD_ALL",
                                     "All students, Undergraduate, Degree/certificate-seeking total"       : "ENROLL_UGRAD_DS",
                                     "All students, Undergraduate, Non-degree/certificate-seeking"         : "ENROLL_UGRAD_NDS"})
                  ## Filling in NA values with 0                   
                  .fillna(0)
)

## Migration

Calculating the percentage of freshmen who are in-state, which is a proxy for regional schools. 

In [48]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_migration = (df_migration
                ## Filtering rows
                .loc[(df_migration["ACAD_YEAR"] == 2023)                    & 
                     (df_migration["UNITID"].isin(df_directory["INST_ID"])) & 
                     (~df_migration["State of residence when student was first admitted"].isin(["All first-time degree/certificate seeking undergraduates, total", "US total"]))]
                ## Subsetting columns
                [["UNITID",
                  "State of residence when student was first admitted",
                  "First-time degree/certificate-seeking undergraduate students"]]
                ## Rename columns
                .rename(columns = {"UNITID"                                                       : "INST_ID",
                                   "State of residence when student was first admitted"           : "STATE",
                                   "First-time degree/certificate-seeking undergraduate students" : "ENROLLMENT"})
)


## aggregating
df_migration['TOTAL_ENROLLMENT'] = df_migration.groupby('INST_ID')['ENROLLMENT'].transform('sum')
df_migration['FTIC_RESD_PCT']   = round(df_migration['ENROLLMENT']/df_migration['TOTAL_ENROLLMENT']*100, 0)


## pulling in some directory data
df_migration = (df_directory
                ## Subsetting columns
                [["INST_ID", 
                  "INST_STATE_NAME"]]
                ## Merging & Filtering
                .merge(df_migration, how = "left", on = "INST_ID")
                .query("STATE == INST_STATE_NAME")
                .drop(["ENROLLMENT", "TOTAL_ENROLLMENT", "INST_STATE_NAME", "STATE"], axis = 1)
)


## Financial Information

Calculating the amount of money spent instruction and research per undergraduate student as well as the size of the endowment per undergraduate. I am also calculating the discount rate, which is a common metric representing the amount of institutional grant aid offered. And finally, I am calculating the percentage of revenue derived from tuition - another common metric that measures the reliance on enrollment.

In [49]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_finance = (df_finance
              ## Filtering rows
              .loc[(df_finance["ACAD_YEAR"] == 2023) & 
                   (df_finance["UNITID"].isin(df_directory["INST_ID"]))]
              ## Subsetting columns
              [["UNITID",
                "Total all revenues and other additions",
                "Instruction - Current year total",
                "Research - Current year total",
                "Tuition and fees, after deducting discounts and allowances",
                "Discounts and allowances applied to tuition and fees",     
                "Scholarships and fellowships expenses -- Current year total",           
                "Value of endowment assets at the beginning of the fiscal year"]]
              ## Renaming columns
              .rename(columns = {"UNITID"                                                        : "INST_ID",
                                 "Total all revenues and other additions"                        : "TOTAL_REV",
                                 "Tuition and fees, after deducting discounts and allowances"    : "TUITION_REV",
                                 "Instruction - Current year total"                              : "INSTRUCT_EXP",
                                 "Research - Current year total"                                 : "RESEARCH_EXP",
                                 "Scholarships and fellowships expenses -- Current year total"   : "SCHOLAR_EXP",
                                 "Discounts and allowances applied to tuition and fees"          : "TUTION_DIS",
                                 "Value of endowment assets at the beginning of the fiscal year" : "ENDOW_VAL"})
               .assign(TOTAL_REV    = lambda x: x['TOTAL_REV'].mask(x['TOTAL_REV'] == 0),
                       INSTRUCT_EXP = lambda x: x['INSTRUCT_EXP'].mask(x['INSTRUCT_EXP'] == 0),
                       ENDOW_VAL    = lambda x: x['ENDOW_VAL'].mask(x['ENDOW_VAL'] == 0))
)


## features creation
df_finance = (df_finance
                .merge(df_enrollment[["INST_ID", "ENROLL_UGRAD_DS"]], on = "INST_ID", how = "left")
                .assign(INSTRUCT_EXP_PER_UG = lambda x: round(x["INSTRUCT_EXP"]/x["ENROLL_UGRAD_DS"], 0),
                        RESEARCH_EXP_PER_UG = lambda x: round(x["RESEARCH_EXP"]/x["ENROLL_UGRAD_DS"], 0),
                        ENDOWMENT_PER_UG    = lambda x: round(x["ENDOW_VAL"]/x["ENROLL_UGRAD_DS"], 0),
                        TUITION_REV_PCT     = lambda x: round(x["TUITION_REV"]/x["TOTAL_REV"]*100, 0),
                        DISCOUNT_RATE       = lambda x: round(x["TUTION_DIS"]/(x["TUITION_REV"] + x["TUTION_DIS"])*100, 0))
                [["INST_ID",
                  "INSTRUCT_EXP_PER_UG",
                  "RESEARCH_EXP_PER_UG",
                  "ENDOWMENT_PER_UG",
                  "TUITION_REV_PCT",
                  "DISCOUNT_RATE"]]
)

## Graduation Rates

Obtaining the four and six year graduation rates for each institution. 

In [50]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_grad_rate = (df_grad_rate
                ## Filtering rows
                .loc[(df_grad_rate["ACAD_YEAR"] == 2023) &
                     (df_grad_rate["UNITID"].isin(df_directory["INST_ID"]))]
                ## Subsetting columns
                [["UNITID",
                  "4-year Graduation rate - bachelor's degree within 100% of normal time",
                  "6-year Graduation rate - bachelor's degree within 150% of normal time"]]
                ## Renaming columns
                .rename(columns = {"UNITID"                                                                : "INST_ID",
                                   "4-year Graduation rate - bachelor's degree within 100% of normal time" : "GRAD_RATE_4YR",
                                   "6-year Graduation rate - bachelor's degree within 150% of normal time" : "GRAD_RATE_6YR"})
)
                


## Student Charges

Obtaining the in-state and out-of-state tuition amounts. 

In [51]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_charges = (df_charges
              ## Filtering rows
              .loc[(df_charges["ACAD_YEAR"] == 2023) &
                   (df_charges["UNITID"].isin(df_directory["INST_ID"]))]
              ## Subsetting columns
              [["UNITID",
                "Published in-state tuition and fees (current year)",
                "Published out-of-state tuition and fees (current year)"]]
              ## Rename columns
              .rename(columns = {"UNITID"                                                 : "INST_ID",
                                 "Published in-state tuition and fees (current year)"     : "TUITION_IN_STATE",
                                 "Published out-of-state tuition and fees (current year)" : "TUITION_OUT_STATE"})
)

## Miscellaneous

Obtaining the retention rate and student-faculty ratio.

In [52]:
## general cleaning - filtering schools, extracting/renaming renaming variables
df_misc = (df_misc
           ## Filter rows
           .loc[(df_misc["ACAD_YEAR"] == 2023) &
                (df_misc["UNITID"].isin(df_directory["INST_ID"]))]
           ## Subsetting columns
           [["UNITID",
             "Student-to-faculty ratio",
             "Full-time retention rate, 2023"]]
           ## Renaming columns
           .rename(columns = {"UNITID"                         : "INST_ID",
                              "Student-to-faculty ratio"       : "STU_FAC_RATIO",
                              "Full-time retention rate, 2023" : "RETAIN_RATE"})
    )

## Compiling Data

Combing all of the data frames into one. 

In [53]:
## merging data
df_master = (df_directory
                 .merge(df_admissions, on = "INST_ID", how = "left")
                 .merge(df_fin_aid,    on = "INST_ID", how = "left")
                 .merge(df_enrollment, on = "INST_ID", how = "left")
                 .merge(df_grad_rate,  on = "INST_ID", how = "left")
                 .merge(df_migration,  on = "INST_ID", how = "left")
                 .merge(df_finance,    on = "INST_ID", how = "left")
                 .merge(df_charges,    on = "INST_ID", how = "left")
                 .merge(df_misc,       on = "INST_ID", how = "left")
)

## getting rid of unneeded objects
del df_directory, df_admissions, df_fin_aid, df_enrollment, df_grad_rate, df_migration, df_finance, df_charges, df_misc

## Missing Values & Final Cleaning

Not every school submits a complete data set, so we have some null values in our final data set. Below, I use the MICE method to impute any missing data and then round the values to the nearest whole number. 

In [ ]:
# Imputing 
imputer    = IterativeImputer(estimator = DecisionTreeRegressor(max_depth = 10), random_state = 0)
df_imputed = pd.DataFrame(imputer.fit_transform(df_master.iloc[:,5:25]), columns = df_master.iloc[:,5:25].columns)
df_imputed = round(df_imputed, 0)


And finally we join the imputed data back onto our list of schools and write the data to file. 

In [55]:
## combinig imputed data with school names/info
df_final = pd.concat([df_master.iloc[:,0:4], df_imputed], axis = 1)

## writing data to file
df_final.to_csv("clean_data.csv", index = False)